# Урок 2. Работа с данными.

In [1]:
import Pkg;
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("DelimitedFiles")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %Fetching: [=====>                                   ]  10.2 %Fetching: [=====>                                   ]  10.9 %Fetching: [=====>                                   ]  11.3 %Fetching: [=====>                                   ]  11.8 %Fetching: [=====>                                   ]  12.0 %Fetching: [=====>                                   ]  12.4 %Fetching: [======>                                  ]  12.7 %Fetching: [======>                                  ]  13.2 %======>                                  ]  14.6 %Fetching: [=======>                                 ]  15.1 %Fetching: [=======>                                 ]  16.4 %Fetching: [========>                                ]  18.6 %Fetching: [========>                                ]  19.0 %Fetching: [=============>     

In [2]:
using CSV  # Чтение и запись CSV файлов
using DataFrames  # Create a data structure
using DelimitedFiles # Работа с файлами с разделителями
using Statistics # Модуль для работы со статистическими данными
# using XLSX

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1273


## 1. Линейная алгебра

In [61]:
A = rand(5,5); # Создадим матрицу размером 5x5
A_t = A' # Транспонирование матрицы
A = A_t*A; # Умножение матриц
A

5×5 Array{Float64,2}:
 0.918589  1.37553  0.772424  1.25694  0.72089
 1.37553   2.7748   1.77415   2.5408   1.83507
 0.772424  1.77415  1.40094   1.85991  1.29512
 1.25694   2.5408   1.85991   2.61212  1.74482
 0.72089   1.83507  1.29512   1.74482  1.96273

- `size()` Покажет размерность матрицы
- `sizeof()` Покажет размер в байтах, а `typeof()` тип данных

In [62]:
size(A), sizeof(A), typeof(A)

((5, 5), 200, Array{Float64,2})

Обращаться к матрице можно по индексу строки и столбца, например так (Обратите внимание, они считаются с 1):

In [63]:
A[2,2]

2.7748028243218226

Или представить, что у нас длинный вектор (аналог операции `flatten`) и задать одним индексом:

In [64]:
A[7]

2.7748028243218226

## 2. Чтение файлов

Возьмем [отсюда](https://www.kaggle.com/shivamb/netflix-shows) датасет и скачаем его, например так:

In [276]:
download("https://raw.githubusercontent.com/VlasovKirill/Julia-in-DS-crash-course/master/data/netflix_titles.csv",
    "netflix.csv")

"netflix.csv"

Для чтения файлов с разделителями можно использовать стандартную библиотеку [DelimitedFiles](https://docs.julialang.org/en/v1/stdlib/DelimitedFiles/) и метода `readdlm`:

In [34]:
netflix, header = DelimitedFiles.readdlm("netflix.csv", ',', header=true)

println("header это: ", typeof(header), " и имеет размерность ", size(header))
println("netflix это: ", typeof(netflix), " и имеет размерность ", size(netflix))

header это: Array{AbstractString,2} и имеет размерность (1, 12)
netflix это: Array{Any,2} и имеет размерность (6234, 12)


В Python мы привыкли работать с табличными данными в виде Dataframe'ов. По аналогии с Pandas, пакет [DataFrames](https://juliadata.github.io/DataFrames.jl/stable/man/getting_started/) позволяет рабоотать с табличными данными придставляя их в виде Dataframe'ов. Объекты типа DataFrame представляют таблицу данных как серию векторов, каждый из которых соответствует столбцу или переменной. 

Создать DataFrame, можно просто передав кажду колонку как вектор: 

In [126]:
df = DataFrame(Month = ["Jan", "Feb", "March"], Value = 1:3)

,Month,Value
,String,Int64
1,Jan,1
2,Feb,2
3,March,3


Посмотрим на тип объекта `df`:

In [127]:
typeof(df)

DataFrame

Теперь можно добавлять столбцы или строки:

In [130]:
old_size = size(df)
df.Year = ones(3)*2020
col_add_size = size(df)
print("фрейм размером ",old_size, " стал ",  col_add_size)
df

фрейм размером (3, 3) стал (3, 3)

,Month,Value,Year
,String,Int64,Float64
1,Jan,1,2020.0
2,Feb,2,2020.0
3,March,3,2020.0


In [137]:
push!(df, ("April", 4, 2020))
print("фрейм размером ",col_add_size, " стал ",  size(df))

фрейм размером (3, 3) стал (10, 3)

Конечно мы можем создать DataFrame и на основе матрицы:

In [140]:
netflix_df = DataFrame(netflix)
first(netflix_df, 3)

,x1,x2,x3,x4
,Any,Any,Any,Any
1,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby"
2,80117401,Movie,Jandino: Whatever it Takes,
3,70234439,TV Show,Transformers Prime,


In [142]:
last(netflix_df, 3)

,x1,x2,x3,x4,x5
,Any,Any,Any,Any,Any
1,80116008,Movie,Little Baby Bum: Nursery Rhyme Friends,,
2,70281022,TV Show,A Young Doctor's Notebook and Other Stories,,"Daniel Radcliffe, Jon Hamm, Adam Godley, Christopher Godwin, Rosie Cavaliero, Vicki Pepperdine, Margaret Clunie, Tim Steed, Shaun Pye"
3,70153404,TV Show,Friends,,"Jennifer Aniston, Courteney Cox, Lisa Kudrow, Matt LeBlanc, Matthew Perry, David Schwimmer"


Лучшим вариантом создать DataFrame может быть прямо при чтении CSV файла c помощью пакета [CSV](https://juliadata.github.io/CSV.jl/stable/)     

In [156]:
netflix_csv = CSV.read("netflix.csv");
typeof(netflix_csv), typeof(netflix_df)

(DataFrame, DataFrame)

In [157]:
first(netflix_csv, 3)

,show_id,type,title,director
,Int64,String,String,String?
1,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby"
2,80117401,Movie,Jandino: Whatever it Takes,missing
3,70234439,TV Show,Transformers Prime,missing


## 3. Срезы и обращение к данным

In [159]:
names(netflix_csv) # Column names can be obtained as strings using the names function:

12-element Array{String,1}:
 "show_id"     
 "type"        
 "title"       
 "director"    
 "cast"        
 "country"     
 "date_added"  
 "release_year"
 "rating"      
 "duration"    
 "listed_in"   
 "description" 

In [160]:
propertynames(netflix_csv) # To get column names as Symbols use the propertynames function:

12-element Array{Symbol,1}:
 :show_id     
 :type        
 :title       
 :director    
 :cast        
 :country     
 :date_added  
 :release_year
 :rating      
 :duration    
 :listed_in   
 :description 

In [161]:
netflix_csv[1:3, :]

,show_id,type,title,director
,Int64,String,String,String?
1,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby"
2,80117401,Movie,Jandino: Whatever it Takes,missing
3,70234439,TV Show,Transformers Prime,missing


In [162]:
netflix_csv[[1, 5, 10], :]

,show_id,type,title,director
,Int64,String,String,String?
1,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby"
2,80125979,Movie,#realityhigh,Fernando Lebrija
3,70304990,Movie,Good People,Henrik Ruben Genz


In [199]:
netflix_csv[[1,2,3,4,5], [:country, :director, :rating, :duration]]

,country,director,rating,duration
,String?,String?,String?,String
1,United States,"Richard Finn, Tim Maltby",TV-PG,90 min
2,United States,missing,TV-MA,94 min
3,United States,missing,TV-Y7-FV,1 Season
4,United States,missing,TV-Y7,1 Season
5,United States,Fernando Lebrija,TV-14,99 min


In [215]:
netflix_csv[netflix_csv.release_year .=2000, [:director, :rating]]

,director,rating
,String?,String?
1,Richard Sears,TV-MA
2,Richard Sears,TV-MA
3,Richard Sears,TV-MA
4,Richard Sears,TV-MA
5,Richard Sears,TV-MA
6,Richard Sears,TV-MA
7,Richard Sears,TV-MA
8,Richard Sears,TV-MA
9,Richard Sears,TV-MA


In [65]:
netflix_csv[netflix_csv.country .="United States", [:director, :rating]]

UndefVarError: UndefVarError: netflix_csv not defined

In [195]:
descr = describe(netflix_csv)
names(descr)

8-element Array{String,1}:
 "variable"
 "mean"    
 "min"     
 "median"  
 "max"     
 "nunique" 
 "nmissing"
 "eltype"  

In [198]:
descr[[:variable, :eltype]]

┌ Warning: `getindex(df::DataFrame, col_inds::Union{AbstractVector, Regex, Not})` is deprecated, use `df[:, col_inds]` instead.
│   caller = top-level scope at In[198]:1
└ @ Core In[198]:1


,variable,eltype
,Symbol,Type
1,show_id,Int64
2,type,String
3,title,String
4,director,"Union{Missing, String}"
5,cast,"Union{Missing, String}"
6,country,"Union{Missing, String}"
7,date_added,"Union{Missing, String}"
8,release_year,Int64
9,rating,"Union{Missing, String}"


## 4. Аггрегация данных

In [66]:
colors = DataFrame(color_id = 1:5, color=["Черный", "Голубой", "Белый", "Желтый", "Амарантово-Пурпурный"])

socks = DataFrame(socks_id = 1:8,
    name = ["Снежана", "Кирилл", "Иосиф", "Инокентий","Африкан","Аркадий", "Маруся", "Вальдемар"],
    color_id = [1,2,1,3,1,5,4,2])

qt_washing = DataFrame(socks_id = [1,2,3,4,5,6,7,8], qt_wash = [10,20,15,42,12,17,27,32])
print()

In [67]:
socks_with_color = innerjoin(socks, colors, on="color_id")

,socks_id,name,color_id,color
,Int64,String,Int64,String
1,1,Снежана,1,Черный
2,2,Кирилл,2,Голубой
3,3,Иосиф,1,Черный
4,4,Инокентий,3,Белый
5,5,Африкан,1,Черный
6,6,Аркадий,5,Амарантово-Пурпурный
7,7,Маруся,4,Желтый
8,8,Вальдемар,2,Голубой


In [68]:
full_info = leftjoin(socks_with_color[:, [:socks_id, :name, :color]], qt_washing, on="socks_id")

,socks_id,name,color,qt_wash
,Int64,String,String,Int64?
1,1,Снежана,Черный,10
2,2,Кирилл,Голубой,20
3,3,Иосиф,Черный,15
4,4,Инокентий,Белый,42
5,5,Африкан,Черный,12
6,6,Аркадий,Амарантово-Пурпурный,17
7,7,Маруся,Желтый,27
8,8,Вальдемар,Голубой,32


In [69]:
grouped = groupby(full_info, :color)

,socks_id,name,color,qt_wash
,Int64,String,String,Int64?
1,1,Снежана,Черный,10
2,3,Иосиф,Черный,15
3,5,Африкан,Черный,12
,socks_id,name,color,qt_wash
,Int64,String,String,Int64?
1,7,Маруся,Желтый,27


In [70]:
combine(grouped, nrow)

,color,nrow
,String,Int64
1,Черный,3
2,Голубой,2
3,Белый,1
4,Амарантово-Пурпурный,1
5,Желтый,1


In [72]:
combine(grouped, nrow, :qt_wash => mean => :mean)

,color,nrow,mean
,String,Int64,Float64
1,Черный,3,12.3333
2,Голубой,2,26.0
3,Белый,1,42.0
4,Амарантово-Пурпурный,1,17.0
5,Желтый,1,27.0


Для более сложных манипуляций с DataFrame'ами существуют разные модули, например: [DataFramesMeta.jl](https://github.com/JuliaData/DataFramesMeta.jl) and [Query.jl](http://www.queryverse.org/Query.jl/stable/). 